⚡ Intermédiaire | ⏱ 45 min | 🔑 Concepts : ruff, mypy, analyse de code

# 01 - Analyse Statique du Code Python

## Objectifs

- Comprendre ce qu'est l'analyse statique et pourquoi elle est essentielle
- Maîtriser **ruff**, le linter et formateur moderne tout-en-un
- Utiliser **mypy** pour la vérification de types statique
- Configurer ces outils dans un projet Python professionnel
- Intégrer l'analyse statique dans votre workflow de développement

## Prérequis

- Python 3.8+
- Connaissance de base de Python
- Comprendre les concepts de fonctions et de types

## 1. Qu'est-ce que l'analyse statique ?

L'**analyse statique** consiste à examiner le code source **sans l'exécuter** pour détecter :

- **Erreurs potentielles** : variables non définies, imports inutilisés, etc.
- **Problèmes de style** : respect de PEP 8, cohérence du code
- **Erreurs de types** : incompatibilités de types de données
- **Code smell** : patterns anti-patterns, complexité excessive

### Avantages

- **Détection précoce** : trouvez les bugs avant l'exécution
- **Qualité** : code plus lisible et maintenable
- **Productivité** : moins de bugs en production
- **Documentation** : les types servent de documentation

## 2. Linting vs Formatting vs Type Checking

### Linting
Vérification des erreurs logiques et du respect des conventions de code.
- Anciens outils : `flake8`, `pylint`
- Moderne : **ruff**

### Formatting
Formatage automatique du code pour une cohérence visuelle.
- Anciens outils : `black`, `isort`
- Moderne : **ruff format**

### Type Checking
Vérification de la cohérence des types de données.
- Outil principal : **mypy**
- Alternatives : `pyright`, `pyre`

## 3. Ruff : Le linter/formateur tout-en-un

**Ruff** est un outil écrit en Rust, extrêmement rapide (10-100x plus rapide que les alternatives).

Il remplace :
- `flake8` (linting)
- `black` (formatting)
- `isort` (tri des imports)
- `pyupgrade` (modernisation du code)
- Et bien d'autres !

In [ ]:
# Installation de ruff
!pip install ruff -q

In [ ]:
# Vérifier la version
!ruff --version

### 3.1 Ruff Check : Linting

In [ ]:
%%writefile example_bad.py
# Code avec des problèmes de style et des erreurs
import os
import sys
import json

def calculer(x,y):
    unused_var = 42
    result= x+y
    return result

def autre_fonction( ):
    liste = [ 1,2,3,4,5 ]
    for i in range(len(liste)):
        print(liste[i])
    
if __name__=="__main__":
    print( calculer(5,10) )

In [ ]:
# Lancer ruff check pour détecter les problèmes
!ruff check example_bad.py

In [ ]:
# Corriger automatiquement ce qui est possible
!ruff check --fix example_bad.py

In [ ]:
# Voir le code corrigé
!cat example_bad.py

### 3.2 Ruff Format : Formatage automatique

In [ ]:
%%writefile example_format.py
# Code mal formaté
def fonction_mal_formatee(param1,param2,param3):
    resultat={"key1":"value1","key2":"value2","key3":"value3","key4":"value4"}
    if param1>10 and param2<20:
        return resultat
    else:
        return None

liste_longue=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]

In [ ]:
# Formater le code
!ruff format example_format.py

In [ ]:
# Voir le résultat
!cat example_format.py

### 3.3 Configuration dans pyproject.toml

Pour un projet professionnel, configurez ruff dans `pyproject.toml` :

In [ ]:
%%writefile pyproject.toml
[tool.ruff]
# Longueur de ligne maximale (défaut : 88 comme black)
line-length = 88

# Version Python cible
target-version = "py38"

# Règles à activer
select = [
    "E",   # pycodestyle errors
    "W",   # pycodestyle warnings
    "F",   # pyflakes
    "I",   # isort (tri des imports)
    "UP",  # pyupgrade (modernisation)
    "B",   # flake8-bugbear (anti-patterns)
    "C4",  # flake8-comprehensions
    "N",   # pep8-naming
    "SIM", # flake8-simplify
]

# Règles à ignorer
ignore = [
    "E501",  # Line too long (géré par le formateur)
]

# Exclure des fichiers/dossiers
exclude = [
    ".git",
    ".venv",
    "__pycache__",
    "build",
    "dist",
]

[tool.ruff.per-file-ignores]
# Ignorer certaines règles dans les tests
"tests/*" = ["S101"]  # Autoriser les assert dans les tests

### 3.4 Principales catégories de règles Ruff

| Code | Description | Exemples |
|------|-------------|----------|
| **E** | Erreurs pycodestyle | E711: `== None` au lieu de `is None` |
| **W** | Warnings pycodestyle | W291: trailing whitespace |
| **F** | Erreurs pyflakes | F401: import non utilisé |
| **I** | isort (imports) | I001: imports mal triés |
| **UP** | pyupgrade | UP006: `List[int]` → `list[int]` (Python 3.9+) |
| **B** | flake8-bugbear | B006: `def func(arg=[]):` (mutable default) |
| **C4** | Comprehensions | C400: `list(generator)` → list comprehension |
| **SIM** | Simplification | SIM108: if-else ternaire |

Voir toutes les règles : https://docs.astral.sh/ruff/rules/

## 4. Mypy : Vérification de types statique

**Mypy** analyse les annotations de types pour détecter les incohérences **avant l'exécution**.

In [ ]:
# Installation de mypy
!pip install mypy -q

In [ ]:
%%writefile example_types.py
# Code avec des annotations de types
def addition(a: int, b: int) -> int:
    return a + b

def saluer(nom: str) -> str:
    return f"Bonjour {nom}"

# Erreurs de types (détectées par mypy)
resultat = addition(5, "10")  # str au lieu de int
message = saluer(123)  # int au lieu de str

# Type inference
x = 10  # mypy infère que x est int
x = "hello"  # Erreur : incompatible type

In [ ]:
# Vérifier les types avec mypy
!mypy example_types.py

### 4.1 Configuration de mypy dans pyproject.toml

In [ ]:
%%writefile pyproject_mypy.toml
[tool.mypy]
# Version Python
python_version = "3.8"

# Strictness (commencer avec ces options)
warn_return_any = true
warn_unused_configs = true
disallow_untyped_defs = false  # Mettre à true progressivement

# Erreurs à afficher
show_error_codes = true
show_column_numbers = true

# Imports
ignore_missing_imports = false
follow_imports = "normal"

# Cache
incremental = true
cache_dir = ".mypy_cache"

# Configuration par module
[[tool.mypy.overrides]]
module = "tests.*"
ignore_errors = true

### 4.2 Mode strict de mypy

Pour un typage très strict (recommandé pour les nouveaux projets) :

In [ ]:
%%writefile example_strict.py
# Exemple avec mode strict
from typing import Optional, List

def process_data(data: List[int]) -> Optional[int]:
    """Retourne la somme si la liste n'est pas vide."""
    if not data:
        return None
    return sum(data)

# Utilisation
result = process_data([1, 2, 3])
if result is not None:
    print(f"Somme: {result}")

In [ ]:
# Vérification en mode strict
!mypy --strict example_strict.py

## 5. Intégration dans VS Code

### Extensions recommandées

1. **Ruff** (charliermarsh.ruff)
   - Linting et formatage en temps réel
   
2. **Pylance** (ms-python.vscode-pylance)
   - Type checking avec pyright (alternative à mypy)
   
3. **Mypy Type Checker** (matangover.mypy)
   - Intégration mypy

### Configuration VS Code (settings.json)

```json
{
  "[python]": {
    "editor.defaultFormatter": "charliermarsh.ruff",
    "editor.formatOnSave": true,
    "editor.codeActionsOnSave": {
      "source.fixAll.ruff": true,
      "source.organizeImports.ruff": true
    }
  },
  "ruff.lint.run": "onType",
  "mypy-type-checker.args": ["--strict"]
}
```

## Pièges courants

### 1. Ignorer systématiquement les warnings

```python
# ❌ MAUVAIS : abus de # noqa
import unused_module  # noqa: F401
x = dangerous_function()  # noqa

# ✅ BON : comprendre et corriger
# Si vraiment nécessaire, être spécifique
import typing  # noqa: F401 - Utilisé dans les annotations de commentaires
```

### 2. Activer mypy strict mode trop tôt

```python
# Progresser graduellement :
# 1. Commencer sans strict
# 2. Ajouter warn_return_any = true
# 3. Ajouter disallow_untyped_defs = true
# 4. Enfin, strict = true
```

### 3. Ne pas configurer les outils

```python
# ❌ MAUVAIS : utiliser les outils sans configuration
# Chaque projet est différent !

# ✅ BON : avoir un pyproject.toml avec les règles de votre équipe
```

### 4. Type checking vs Runtime validation

```python
# ⚠️ ATTENTION : mypy ne vérifie PAS à l'exécution
def process(x: int) -> int:
    return x * 2

# Ceci s'exécutera sans erreur (mais mypy le détecte)
result = process("hello")  # Runtime TypeError

# Pour validation runtime, utiliser pydantic ou typeguard
```

## Mini-Exercices

### Exercice 1 : Configurer ruff pour un projet

Créez un fichier `pyproject.toml` avec une configuration ruff qui :
- Active les règles E, W, F, I, UP, B, SIM
- Ignore E501 (line too long)
- Cible Python 3.10+
- Exclut le dossier `migrations/`

In [ ]:
# Votre solution ici


### Exercice 2 : Corriger des erreurs de linting

Corrigez le code suivant pour qu'il passe ruff check sans erreurs :

In [ ]:
%%writefile exercice2.py
import sys
import os
import json

def calculer_moyenne(nombres):
    total = 0
    for i in range(len(nombres)):
        total = total + nombres[i]
    return total / len(nombres)

def filtrer_pairs(liste):
    resultat = []
    for element in liste:
        if element % 2 == 0:
            resultat.append(element)
    return resultat

In [ ]:
# Vérifier les erreurs
!ruff check exercice2.py

In [ ]:
# Votre solution corrigée ici


### Exercice 3 : Ajouter des annotations de types

Ajoutez des annotations de types complètes au code suivant et vérifiez avec mypy :

In [ ]:
%%writefile exercice3.py
# Ajoutez les types
def trouver_max(nombres):
    if not nombres:
        return None
    max_val = nombres[0]
    for num in nombres:
        if num > max_val:
            max_val = num
    return max_val

def formater_resultat(valeur):
    if valeur is None:
        return "Aucune valeur"
    return f"Maximum: {valeur}"

In [ ]:
# Votre solution avec types ici


## Solutions

### Solution Exercice 1

In [ ]:
%%writefile pyproject_ex1.toml
[tool.ruff]
line-length = 88
target-version = "py310"

select = [
    "E",   # pycodestyle errors
    "W",   # pycodestyle warnings
    "F",   # pyflakes
    "I",   # isort
    "UP",  # pyupgrade
    "B",   # flake8-bugbear
    "SIM", # flake8-simplify
]

ignore = ["E501"]

exclude = [
    ".git",
    ".venv",
    "__pycache__",
    "migrations",
]

### Solution Exercice 2

In [ ]:
%%writefile exercice2_solution.py
# Imports inutilisés supprimés
# Boucles simplifiées avec sum() et list comprehension

def calculer_moyenne(nombres):
    return sum(nombres) / len(nombres)

def filtrer_pairs(liste):
    return [element for element in liste if element % 2 == 0]

### Solution Exercice 3

In [ ]:
%%writefile exercice3_solution.py
from typing import Optional

def trouver_max(nombres: list[int]) -> Optional[int]:
    if not nombres:
        return None
    max_val = nombres[0]
    for num in nombres:
        if num > max_val:
            max_val = num
    return max_val

def formater_resultat(valeur: Optional[int]) -> str:
    if valeur is None:
        return "Aucune valeur"
    return f"Maximum: {valeur}"

In [ ]:
# Vérification mypy
!mypy exercice3_solution.py